In [26]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import pickle as pkl

pd.set_option('display.max_rows', 500)

In [31]:
# input content file
CONTENT_FILE = '/Users/sjlee/Documents/github/ncses_data/2022-06-29-prod-cms-content.xlsx'
sections = pd.read_excel(CONTENT_FILE, sheet_name='Sections')
pubs = pd.read_excel(CONTENT_FILE, sheet_name='Publications')

In [32]:
# filter to just the urls that need to be coded
typ = ['ONE', 'WP', 'REF']
sections = content[content['Pub Type'].isin(typ)]
pubs = pubs[pubs['Type'].isin(typ)]

In [33]:
sections['URL-base'] = sections['URL'].apply(lambda url: url.split('#')[0])
pubs['URL-base'] = pubs['URL']
sections = pd.merge(sections, pubs[['Publish Date', 'URL-base']], on='URL-base')

/Users/sjlee/opt/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
cols = ['Legacy', 'Pub Type', 'Asset Type', 'Publish Date', 'Pub ID', 'Path', 'Title', 'URL', 'Text']
parsed_lst = []

for url_base in set(sections['URL-base']):
    page = requests.get(url_base)
    soup = BeautifulSoup(page.content, "html.parser")
    sec = sections[sections['URL-base']==url_base]
    for i, row in sec.iterrows():
        legacy = 'N'
        pub_type = row['Pub Type']
        asset_type = 'text'
        pub_date = row['Publish Date']
        pub_id = row['Section ID']
        slug_path = row['Slug Path']
        title = row['Title']
        url_link = row['URL']
        
        slug = slug_path.split('->')[1:]
        if len(slug) == 1:
            sid = slug[0]
        else: 
            sid = ('_').join(slug)
            
        tab_soup = soup.find('section', id=sid)
        if tab_soup != None: 
            para = tab_soup.find_all('p')
            if len(para) > 0:
                text = ' '.join([p.text for p in para])
            else:
                text = ''
        else:
            text = ''
        
        return_lst = [legacy, pub_type, asset_type, pub_date, pub_id, slug_path, title, url_link, text]
        parsed_lst.append(return_lst)


In [50]:
df = pd.DataFrame(parsed_lst, columns=cols)

In [53]:
df.to_csv('/Users/sjlee/Documents/github/ncses_data/current_scraped_files/cms_other.csv', index = False)